In [ ]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
pf = pd.read_csv('data/FEdata.csv')

In [ ]:
pf = pf.drop(['Over18','StandardHours'],axis=1)

In [ ]:
pf.head()

In [ ]:
pf.info()

In [ ]:
y_train = torch.from_numpy(np.array(pf['Attrition']))

In [ ]:
y_train.type()

In [ ]:
y_train.size()

In [ ]:
y_train = y_train.reshape(1100,1)

In [ ]:
y_train = y_train.float()

In [ ]:
pf = pf.drop(['Attrition'],axis=1)

In [ ]:
pfnp = np.array(pf)

In [ ]:
pfnp.dtype

In [ ]:
pfnp = pfnp/pfnp.max(axis=0)
pfnp

In [ ]:
x_train = torch.from_numpy(pfnp)

In [ ]:
x_train.size()

In [ ]:
x_train = x_train.float()
x_train.type()

In [ ]:
x_train

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(28,128)
        self.l2 = nn.Linear(128,32)
        self.l3 = nn.Linear(32,1)
        
    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = torch.sigmoid(self.l3(x))
        return x
    
net = Net()

print(net)

In [ ]:
test = net(x_train)
print(test.size(),test.type,y_train.size(),y_train.type())

In [ ]:
optimizer = torch.optim.Adam(net.parameters(), lr=0.0001)
loss_func = torch.nn.MSELoss() 

for i in range(1000):
    out = net(x_train)
    loss = loss_func(out,y_train)
    optimizer.zero_grad()   # 清空上一步的残余更新参数值
    loss.backward()         # 误差反向传播, 计算参数更新值
    optimizer.step()        # 将参数更新值施加到 net 的 parameters 上

In [ ]:
pf_pre = pd.read_csv('data/FEtest.csv')
pf_pre = pf_pre.drop(['Over18','StandardHours'],axis=1)
pfnp_pre = np.array(pf_pre)
pfnp_pre = pfnp_pre/pfnp_pre.max(axis=0)
x_pre = torch.from_numpy(pfnp_pre)
x_pre.size()

In [ ]:
x_pre = x_pre.float()
pre = net(x_pre)

In [ ]:
pre

In [ ]:
pre = pre.detach().numpy()

In [ ]:
len(pre)

In [ ]:
for x in range(len(pre)):
    if pre[x]>0.5:
        pre[x]=1
    else:
        pre[x]=0
pre

In [ ]:
out = pd.DataFrame(pre,dtype='int64')

In [ ]:
out.head(1100)

In [ ]:
out.columns=['result']

In [ ]:
out.head()

In [ ]:
out.to_csv('result2.csv',index=False)